In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import networkx as nx
import numpy as np
from matrix_to_network import CNN

## Sample Architectures for testing

### Arch 1

### Arch 2

### Arch 3